# SO, SO2, PPD: comunicação e sincronização de *threads*

Hélio - DC/UFSCar - 2023

# Sincronização de *threads*

*Threads* compartilham o espaço de endereçamento do processo ao qual estão associadas. Isso facilita as comunicações entre elas, uma vez que quaisquer estruturas manipuladas de maneira particionada podem ser posicionadas em áreas de memória comuns, acessíveis por todas as *threads*.

Por outro lado, a consistência dos dados compartilhados pode depender de certos acessos ocorrerem de **maneira exclusiva** e é claro que essa sincronização não seria eficiente se fosse implementada com algum tipo de ***busy waiting***, salvo em bloqueios de curtíssima duração.

```
while(recurso_ocupado())
  fica_aqui_à_toa_gastando_fatia_de_tempo_até_outra_tarefa_liberá-lo();
```

Assim, a ideia é ter mecanismos de sincronização e exclusão mútua que possibilitem que as tarefas sejam automaticamente suspensas quando condições necessárias não são satisfeitas. Uma tarefa que aguarda uma condição pode ser tirada de execução e outra tarefa pode ser selecionada para ocupar o processador que se tornou disponível agora.

O primeiro passo para isso é ter uma biblioteca padronizada que ofereça esses serviços. No caso do uso de *threads* com a biblioteca ***pthreads***, há uma série de estruturas de sincronização, como pode ser visto em [pthread.h](https://man7.org/linux/man-pages/man0/pthread.h.0p.html).

É útil para a aplicação usar bibliotecas padronizadas, já que isso aumenta a portabilidade dos programas.

* No caso específico do sistema Linux, por razões de eficiência, as implementações dessas bibliotecas procuram fazer todas as operações possíveis no "espaço de usuário" (*user space*), evitando o custo dos mecanismos de chamada de sistema. Isto é importante, já que a eficiência dos mecanismos de sincronização é fundamental para o bom desempenho das aplicações.

* Quando o apoio do SO (Linux) é necessário para as sincronizações, a chamada [futex](https://man7.org/linux/man-pages/man2/futex.2.html) pode ser usada, por exemplo, tanto para bloquear (***FUTEX_WAIT***), quanto para acordar (***FUTEX_WAKE***) tarefas.

Quando vai bloquear uma tarefa, o SO primeiro ajusta e salva informações de contexto da tarefa que emite a chamada. O descritor desta tarefa é então "desligado" da lista de tarefas prontas e inserido na fila das tarefas que aguardam que essa condição seja satisfeita. Tipicamente, a tarefa é inserida na fila de tarefas associada ao semáforo ou mutex em questão (ou futex, no Linux). O SO segue selecionando outra tarefa pronta para execução e restaurando seu contexto para que ela volte a ser executada.

Pensando que estamos estudando a lógica dos mecanismos de sincronização e que queremos criar programas portáteis, sigamos usando as APIs POSIX e os mecanismos providos para isto!

Faz sentido?



# Mutexes

O exemplo de código a seguir ilustra o uso de uma estrutura **mutex** para sincronização, garantindo a exclusão mútua entre as *threads* na manipulação de um *buffer* compartilhado.

Inernamente, um mutex tem uma variável *booleana* que indica se o recurso está livre ou ocupado.

Além disso, há uma fila de tarefas onde serão inseridas as tarefas que tentarem bloquer o *mutex* quando ele já estiver bloqueado.

As operações básicas em mutex são as seguintes:

* int [pthread_mutex_init](https://man7.org/linux/man-pages/man3/pthread_mutex_init.3p.html)(pthread_mutex_t *restrict mutex, const pthread_mutexattr_t *restrict attr);

    // ou:
    pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;

* int pthread_mutex_destroy(pthread_mutex_t *mutex);

* int [pthread_mutex_lock](https://man7.org/linux/man-pages/man3/pthread_mutex_lock.3p.html)(pthread_mutex_t *mutex);

* int [pthread_mutex_unlock](https://man7.org/linux/man-pages/man3/pthread_mutex_unlock.3p.html)(pthread_mutex_t *mutex);

* int pthread_mutex_trylock(pthread_mutex_t *mutex);



Em geral, uma aplicação que usa um mutex deve iniciá-lo, indicando que o recurso associado ao mutex está disponível.

Ao longo de suas operações, tarefas que precisam usar um recurso com exclusão mútua emitem uma chamada pthread_mutex_lock, seguida de pthread_mutex_unlock depois que liberarem o recurso que precisa ser usado com exclusão mútua.

In [ ]:
%%writefile m1.c
/*
** Objetivo: uso de mutex para prover exclusão mútua
**
** Neste exemplo, múltiplas tarefas precisam incrementar uma variável global.
   Para que o resultado não fique inconsistente, apenas 1 tarefa deve
   incrementá-la de cada vez. Para garantir essa exclusão mútua, cada uma
   deve "cercar" a manipulação da variável com as chamadas de bloqueio e
   liberação do mutex.
*/

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <pthread.h>

/*
A  mutex  is  a  MUTual  EXclusion device, and is useful for protecting
shared data structures from concurrent modifications, and  implementing
critical sections and monitors.

A  mutex  has  two possible states: unlocked (not owned by any thread),
and locked (owned by one thread). A mutex can never  be  owned  by  two
different  threads  simultaneously. A thread attempting to lock a mutex
that is already locked by another thread is suspended until the  owning
thread unlocks the mutex first.

!pthread_mutex_init! initializes the mutex object pointed to by |mutex|
according to the mutex attributes specified in |mutexattr|.  If |mutex-
attr| is !NULL!, default attributes are used instead.
*/

#define N_THREADS 3
#define TTL			3
#define LEN 128


// declaracao do mutex deve ser global

pthread_mutex_t _mutex = PTHREAD_MUTEX_INITIALIZER; // criacao e inicializacao estatica
// ou
// pthread_mutex_t _mutex;                          // declaracao sem iniciacao
// pthread_mutex_init(&_mutex, NULL);               // inicializar na fun?ao main

int _result = 0;

void *
soma(void *arg)
{
	int i;
	long int th_num = (long int)arg;

	for (i=0; i<TTL; i++) {
    // dorme por até 2 segundos
    sleep (rand()%3);

		printf("Th %ld esperando...\n",th_num); fflush(stdout);

		pthread_mutex_lock(&_mutex);

	  	printf("Th: %ld na secao critica\n",th_num); fflush(stdout);
		  _result = _result + th_num;
		  sleep(1);
  		printf("Th: %ld liberando secao critica\n",th_num); fflush(stdout);

		pthread_mutex_unlock(&_mutex);
	}

	pthread_exit(NULL);
}


int
main (int argc, char *argv[])
{
	pthread_t thread[N_THREADS];
	int result;
	long int t;
	char err_msg[LEN];

	// cria threads
	for(t=0; t<N_THREADS; t++) {
		// printf("Criando thread %d\n", t);
		if((result=pthread_create(&thread[t], NULL, soma, (void *)t))) {
			strerror_r(result,err_msg,LEN);
			printf("Erro criando thread: %s\n",err_msg);
			exit(0);
		}
	}

	// espera threads conclu?rem
	for(t=0; t<N_THREADS; t++) {
		if((result=pthread_join(thread[t], NULL))) {
			strerror_r(result,err_msg,LEN);
			printf("Erro em pthread_join: %s\n",err_msg);
			exit(0);
		}
	}

	printf("Soma final: %d\n",_result);

	// libera mutex
	pthread_mutex_destroy (&_mutex);

	return EXIT_SUCCESS;
}

Writing m1.c


In [ ]:
! if [ ! m1 -nt m1.c ]; then gcc -Wall m1.c -o m1 -pthread; fi
! ./m1

Th 2 esperando...
Th: 2 na secao critica
Th 1 esperando...
Th 0 esperando...
Th: 2 liberando secao critica
Th: 1 na secao critica
Th: 1 liberando secao critica
Th: 0 na secao critica
Th 2 esperando...
Th: 0 liberando secao critica
Th: 2 na secao critica
Th 1 esperando...
Th 0 esperando...
Th: 2 liberando secao critica
Th: 1 na secao critica
Th: 1 liberando secao critica
Th: 0 na secao critica
Th 2 esperando...
Th 1 esperando...
Th: 0 liberando secao critica
Th: 2 na secao critica
Th 0 esperando...
Th: 2 liberando secao critica
Th: 1 na secao critica
Th: 1 liberando secao critica
Th: 0 na secao critica
Th: 0 liberando secao critica
Soma final: 9


# Semáforos

Outra estrutura disponível em APIs POSIX para sincronização entre *threads* são os **semáforos**.

Logicamente, um semáforo consiste de um **contador** e uma **fila de tarefas** que estiverem bloquedas à espera de poder **decrementar** o valor do contador do semáforo sem que ele fique negativo.

As principais funções para manipulação de semároforos com a API POSIX incluem:


// Initialize semaphore object SEM to VALUE.
* int [sem_init](https://man7.org/linux/man-pages/man3/sem_init.3p.html) (sem_t *sem, int pshared, unsigned int value);

// Free resources associated with semaphore object SEM.
* int sem_destroy (sem_t *sem);

// Post SEM.
* int [sem_post](https://man7.org/linux/man-pages/man3/sem_post.3p.html) (sem_t *sem);

// Wait for SEM being posted.
* int [sem_wait](https://man7.org/linux/man-pages/man3/sem_wait.3p.html) (sem_t *sem);

O exemplo a seguir ilustra o uso da API **POSIX *semaphores*** em um modelo de programa do tipo produtor/consumidor.

Há várias soluções para esse tipo de problema, dependendo do número de produtores e de consumidores. Se houver um *buffer* compartilhado, é comum que também seja usada uma estrutura *mutex* para garantir a exclusão mútua no acesso a esse recurso.

A ideia de ter semáforos com contadores de recursos associados é permitir que diversas operações sejam automaticamente liberadas, ao invés de implementar-se um mecanismo de alternância entre os atores que produzem e liberam recursos.

Na implementação exeplo apresentada a seguir, são usados dois semáforos. Um deles indica quantos espaços estão disponíveis no *buffer* para inserção de itens produzidos. É claro que esse semáforo deve ser iniciado com um número que representa o tamanho do *buffer*.

O outro semáforo indica quantos itens produzidos há no *buffer*. O contador desse semáforo deve ser iniciado com 0 e será incrementado cada vez que o produtor inserir algo no *buffer* e emitir a chamada de liberação do semáforo - sem_post(). O decremento ocorrerá cada vez que o consumidor emitir a chamada sem_wait().

Com isso, nesse caso é possível ter um comportamento que suporta variações na velocidade de produção e na velocidade de consumo. O *buffer* acomodará itens produzidos e ainda não consumidos.

In [ ]:
%%writefile s1.c

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <pthread.h>
#include <semaphore.h>
#include <string.h>
#include <errno.h>


#define N_TIMES	5
#define MAX 128

/*
// System specific semaphore definition.
typedef struct {
  struct _pthread_fastlock __sem_lock;
  int __sem_value;
  _pthread_descr __sem_waiting;
} sem_t;
*/

/*
// Value returned if `sem_open' failed.
#define SEM_FAILED	((sem_t *) 0)

// Maximum value the semaphore can have.
#define SEM_VALUE_MAX (2147483647)

// Initialize semaphore object SEM to VALUE. If PSHARED then share it
// with other processes.
int sem_init (sem_t *sem, int pshared, unsigned int value);

// Free resources associated with semaphore object SEM.
int sem_destroy (sem_t *sem);

// Open a named semaphore NAME with open flaot OFLAG.
sem_t *sem_open (const char *name, int oflag, ...);

// Close descriptor for named semaphore SEM.
int sem_close (sem_t *sem);

// Remove named semaphore NAME.
int sem_unlink (const char *name);

// Wait for SEM being posted.
int sem_wait (sem_t *sem);

// Similar to `sem_wait' but wait only until ABSTIME.
int sem_timedwait (sem_t *restrict sem, const struct timespec *restrict abstime);

// Test whether SEM is posted.
int sem_trywait (sem_t *sem);

// Post SEM.
int sem_post (sem_t *sem);

// Get current value of SEM and store it in *SVAL.
int sem_getvalue (sem_t *restrict sem, int *restrict sval);
*/

/*
	POSIX 1003.1b semaphores; not to be  confused  with
	SystemV semaphores as described in !ipc!(5), !semctl!(2) and !semop!(2).

	Semaphores are counters for resources shared between threads. The basic opera-
	tions on semaphores are: increment the counter atomically, and wait until  the
	counter is non-null and decrement it atomically.

	!sem_init!  initializes  the  semaphore  object pointed to by |sem|. The count
	associated with the semaphore is set initially to |value|. The |pshared| argu-
	ment  indicates  whether  the  semaphore  is  local  to  the  current  process
	(|pshared| is zero) or is to be shared between several processes (|pshared| is
	not  zero). LinuxThreads currently does not support process-shared semaphores,
	thus !sem_init! always returns with error !ENOSYS! if |pshared| is not zero.

	!sem_wait! suspends the calling thread until the semaphore pointed to by |sem|
	has non-zero count. It then atomically decreases the semaphore count.

	!sem_trywait!  is  a  non-blocking  variant  of  !sem_wait!.  If the semaphore
	pointed to by |sem| has non-zero count, the count is atomically decreased  and
	!sem_trywait!  immediately  returns  0.   If  the  semaphore  count  is  zero,
	!sem_trywait! immediately returns with error !EAGAIN!.

	!sem_post! atomically increases the count  of  the  semaphore  pointed  to  by
	|sem|.  This function never blocks and can safely be used in asynchronous sig-
	nal handlers.

	!sem_getvalue! stores in the location pointed to by |sval| the  current  count
	of the semaphore |sem|.

	!sem_destroy!  destroys  a  semaphore  object,  freeing the resources it might
	hold. No threads should be waiting on the semaphore at the time  !sem_destroy!
	is  called.  In  the  LinuxThreads implementation, no resources are associated
	with semaphore objects,  thus  !sem_destroy!   actually  does  nothing  except
	checking that no thread is waiting on the semaphore.
*/


sem_t s_prod, s_cons; // semaforos

int _val;


void *
produz(void *n_times)
{
	int i;

	for (i=0; i<(long int)n_times; i++) {
		sem_wait(&s_prod);
		_val++;
		printf("Produziu %d\n",_val); fflush(stdout);
		sem_post(&s_cons);
	}
	pthread_exit(NULL);
}

void *
consome(void *n_times)
{
	int i;

	for (i=0; i<(long int)n_times; i++) {
		sem_wait(&s_cons);
		printf("Consumiu: %d\n",_val);
		sleep(random()%5);
		sem_post(&s_prod);
	}
	pthread_exit(NULL);
}


int
main()
{
	pthread_t th_prod, th_cons;
	char err_msg[MAX];

	_val = 0;

	// char *strerror_r(int errnum, char *buf, size_t buflen);

	// int sem_init (sem_t *sem, int pshared, unsigned int value);
	if (sem_init(&s_cons, 0, 0) < 0) {
		strerror_r(errno,err_msg,MAX);
		printf("Erro em sem_init: %s\n",err_msg);
		exit(1);
	}

	// int sem_init (sem_t *sem, int pshared, unsigned int value);
	if (sem_init(&s_prod, 0, 1) < 0) {
		strerror_r(errno,err_msg,MAX);
		printf("Erro em sem_init: %s\n",err_msg);
		exit(1);
	}
	if (pthread_create(&th_prod, NULL, produz, (void *)N_TIMES) != 0) {
		strerror_r(errno,err_msg,MAX);
		printf("Erro em pthread_create: %s\n",err_msg);
		exit(1);
	}
	if (pthread_create(&th_cons, NULL, consome, (void *)N_TIMES) != 0) {
		strerror_r(errno,err_msg,MAX);
		printf("Erro em pthread_create: %s\n",err_msg);
		exit(1);
	}

	pthread_join(th_prod, NULL);
	pthread_join(th_cons, NULL);

	sem_destroy(&s_prod);
	sem_destroy(&s_cons);

	return(0);
}

Overwriting s1.c


In [ ]:
! if [ ! s1 -nt s1.c ]; then gcc -Wall s1.c -o s1 -pthread; fi
! ./s1

Produziu 1
Consumiu: 1
Produziu 2
Consumiu: 2
^C


# Barreiras

Outra estrutura de sincronização fornecida por APIs POSIX é a **barreira**.
Trata-se de um mecanismo que faz com que as *threads* de um grupo só prossigam em execução após **todas** as *threads* deste grupo fazerem a chamada de sincronização na barreira.

Algumas das funções mais usuais na manipulação de barreiras incluem:
```
// Initialize BARRIER with the attributes in ATTR.  The barrier is
// opened when COUNT waiters arrived.
int pthread_barrier_init(pthread_barrier_t *barrier, const pthread_barrierattr_t *attr, unsigned count);

// Wait on barrier BARRIER
int pthread_barrier_wait(pthread_barrier_t *barrier);

// Destroy a previously dynamically initialized barrier BARRIER.
int pthread_barrier_destroy(pthread_barrier_t *barrier);
```
Para usar a barreira, inicialmente ela deve ser ajustada com o número de *threads* que vão sincronizar-se com ela. À medida que cada *thread* realiza a operação de sincronização na barreira, este contador é decrementado. Enquanto seu valor não atingir 0, as *threads* que fizerem a chamada ficam bloqueadas numa fila associada à barreira. Quando a última *thread* fizer a chamada de sincronização, esta e todas as *threads* que aguardavam são desbloqueadas e podem prosseguir em execução.

In [ ]:
%%writefile t3.c

#include <pthread.h>
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <errno.h>
#include <string.h>


#ifndef PTHREAD_BARRIER_SERIAL_THREAD
	#define PTHREAD_BARRIER_SERIAL_THREAD -1
#endif

#define NUM_THREADS	4
#define LEN 128

#if defined(__GNU_LIBRARY__) && !defined(_SEM_SEMUN_UNDEFINED)
#else
// POSIX barriers data type. The structure of the type is deliberately not exposed.
typedef union {
  char __size[__SIZEOF_PTHREAD_BARRIER_T];
  long int __align;
} pthread_barrier_t;

typedef union {
	char __size[__SIZEOF_PTHREAD_BARRIERATTR_T];
	int __align;
} pthread_barrierattr_t;
#endif

// Functions to handle barriers

// Initialize BARRIER with the attributes in ATTR.  The barrier is
// opened when COUNT waiters arrived.
int pthread_barrier_init(pthread_barrier_t *barrier, const pthread_barrierattr_t *attr, unsigned count);

// Destroy a previously dynamically initialized barrier BARRIER.
int pthread_barrier_destroy(pthread_barrier_t *barrier);

// Wait on barrier BARRIER
int pthread_barrier_wait(pthread_barrier_t *barrier);

// Initialize barrier attribute ATTR
int pthread_barrierattr_init(pthread_barrierattr_t *attr);

// Destroy previously dynamically initialized barrier attribute ATTR.
int pthread_barrierattr_destroy(pthread_barrierattr_t *attr);

// Get the process-shared flag of the barrier attribute ATTR
int pthread_barrierattr_getpshared(const pthread_barrierattr_t *attr, int *pshared);

// Set the process-shared flag of the barrier attribute ATTR.
int pthread_barrierattr_setpshared(pthread_barrierattr_t *attr, int pshared);


pthread_barrier_t _barrier;

/*
	int pthread_barrier_wait(pthread_barrier_t *barrier);

	The  pthread_barrier_wait()  function shall synchronize participating threads at
	the barrier referenced by barrier.  The calling thread shall block until the
	required number of threads have  called pthread_
	barrier_wait() specifying the barrier.

	When  the required number of threads have called pthread_barrier_wait() specifying
	the barrier, the constant PTHREAD_BARRIER_SERIAL_THREAD shall be returned to one
	unspecified thread and  zero  shall be  returned  to  each  of  the remaining
	threads. At this point, the barrier shall be reset to the state it had as a result
	of the most recent pthread_barrier_init() function that referenced it.

	RETURN VALUE
	Upon  successful  completion,  the  pthread_barrier_wait()  function  shall return
	PTHREAD_BARRIER_SERIAL_THREAD for a single (arbitrary) thread synchronized at the
	barrier and zero for each of the other threads. Otherwise, an error number shall
	be returned to indicate the error.
*/

void *hello_w(void *thread_num)
{
	int i,t, result;
	char err_msg[LEN];

for(i=0;i<3;i++) {

	t=random()%10;
	printf("Th %ld vai dormir %d s\n",(long)thread_num,t);
	sleep(t);

	// int pthread_barrier_wait(pthread_barrier_t *barrier);
	result=pthread_barrier_wait(&_barrier);

	if(result>0) {
		strerror_r(result,err_msg,LEN);
		printf("Th %ld - erro em barrier_wait: %s\n",(long)thread_num,err_msg);
	}else {
		// printf("Th %ld - resultado de barrier_wait: ",(long)thread_num);
		if(result==PTHREAD_BARRIER_SERIAL_THREAD) {
			printf("PTHREAD_BARRIER_SERIAL_THREAD\n");
			fflush(stdout);
		} else {
			printf("%d\n",result);
			fflush(stdout);
		}
	}
	printf("Thread %ld depois da barreira\n",(long)thread_num);
	fflush(stdout);
} // for

	pthread_exit(NULL);
}

int
main()
{
	pthread_t threads[NUM_THREADS];
	int status;
	long t;
	char err_msg[LEN];

	srandom(getpid());
/*
	// int pthread_barrier_init(pthread_barrier_t *restrict barrier,
	                            const pthread_barrierattr_t *restrict attr, unsigned count);

	The  pthread_barrier_init() function shall allocate any resources required to use the barrier referenced
	by barrier and shall initialize the barrier with attributes referenced by attr.  If attr  is  NULL,  the
	default  barrier  attributes  shall  be used; the effect is the same as passing the address of a default
	barrier attributes object. The results are undefined if pthread_barrier_init() is called when any thread
	is  blocked on the barrier (that is, has not returned from the pthread_barrier_wait() call). The results
	are undefined if a barrier is used without  first  being  initialized.  The  results  are  undefined  if
	pthread_barrier_init() is called specifying an already initialized barrier.

	The count argument specifies the  number  of  threads  that  must  call
	pthread_barrier_wait()  before any of them successfully return from the
	call.  The value specified by count must be greater than zero.
*/
 	pthread_barrier_init(&_barrier, NULL, NUM_THREADS);

	for(t=0; t<NUM_THREADS; t++){

		status = pthread_create(&threads[t], NULL, hello_w, (void *)t);
		if (status) {
			strerror_r(errno,err_msg,LEN);
			printf("Falha da criacao da thread %ld (%d): %s\n",t,status,err_msg);
			exit(EXIT_FAILURE);
		}
	}

	for(t=0; t<NUM_THREADS; t++) {
		pthread_join(threads[t],NULL);
	}

/*
	// int pthread_barrier_destroy(pthread_barrier_t *barrier);

	The pthread_barrier_destroy() function shall destroy the barrier referenced by barrier and  release  any
	resources  used  by the barrier. The effect of subsequent use of the barrier is undefined until the bar-
	rier is reinitialized by another call to pthread_barrier_init(). An implementation may use this function
	to  set  barrier  to  an invalid value. The results are undefined if pthread_barrier_destroy() is called
	when any thread is blocked on the barrier, or if this function is called with an uninitialized  barrier.
*/

 	pthread_barrier_destroy(&_barrier);

	return(0);
}

Overwriting t3.c


In [ ]:
!gcc t3.c -o t3 -pthread && ./t3

Th 0 vai dormir 6 s
Th 2 vai dormir 5 s
Th 1 vai dormir 7 s
Th 3 vai dormir 0 s
PTHREAD_BARRIER_SERIAL_THREAD
Thread 1 depois da barreira
Th 1 vai dormir 0 s
0
Thread 3 depois da barreira
Th 3 vai dormir 9 s
0
Thread 2 depois da barreira
Th 2 vai dormir 1 s
0
Thread 0 depois da barreira
Th 0 vai dormir 7 s
PTHREAD_BARRIER_SERIAL_THREAD
Thread 3 depois da barreira
Th 3 vai dormir 0 s
0
Thread 1 depois da barreira
Th 1 vai dormir 2 s
0
Thread 2 depois da barreira
Th 2 vai dormir 5 s
0
Thread 0 depois da barreira
Th 0 vai dormir 9 s
PTHREAD_BARRIER_SERIAL_THREAD
Thread 0 depois da barreira
0
Thread 2 depois da barreira
0
Thread 1 depois da barreira
0
Thread 3 depois da barreira


Outras estruturas e APIs para comunicação e sincronização de *threads* incluem as variáveis de condição (*condition variables*), os bloqueios de leitura e escrita (*reader/writer locks*), e *spins*.